In [ ]:
### НУЖНЫЕ ИМПОРТЫ
# !pip install python-dotenv==1.0.1 mistralai

## 1. USER QUERY

## 2. RETRIEVAL
#### 2.1 BUILDING QUERY EMBEDDINGS

#### 2.2 CHANKING DATA

наверное нужно все данные заранее сделать, а не делать это все в онлайне, просто BERT офигеет столько данных через себя прогонять

#### 2.3 SEARCH

## 3. PROMPT BUILDING

## 4. LLM ANSWER

In [ ]:
from TerrariaRAG import TerrariaRAG

from agent import MistralLLM, CraftAgent, GeneralAgent
from mistralai import Mistral
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings

import os
import json

load_dotenv()

client = MistralLLM(
    mistral_client=Mistral(os.getenv("API_KEY")),
    model_name="mistral-small-latest"
)

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

recipes = json.load(open("../data/recipes.json", "r", encoding="utf-8"))

craft_agent = CraftAgent(
    name="CraftAgent",
    llm_session=client,
    recipes=recipes,
    embeddings=embeddings,
    max_recipes=5
)

general_agent = GeneralAgent(
    name="GeneralAgent",
    llm_session=client,
    embeddings=embeddings,
    max_docs=5
)

terraria_rag = TerrariaRAG(
    llm_session=client,
    agents=[
        craft_agent, 
        general_agent
    ]
)

In [ ]:
question = (
    "Что такое крылья ангела, зачем они нужны и как их создать? "
    "А ещё как скрафтить золотой меч?"
)
response = terraria_rag.run(question)
print("=" * 50)
print("Вопрос:", question, "\n", "-" * 40, "\n")
print("Ответ:", response)